In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import sqlite3

In [2]:
creates = create_engine("sqlite:///shops.db")
db_path = "shops.db"

In [ ]:
def ingest_db(file_path, table_name, engine):
    df = pd.read_csv(file_path)
    df.to_sql(table_name, con=creates, if_exists="replace", index=False)
    print(df.shape)

data_folder = "C:/Users/amanp/Downloads/data/data/"
for file in os.listdir(data_folder):
    if file.endswith('.csv'):
        file_path = os.path.join(data_folder, file)
        table_name = os.path.splitext(file)[0]  # Removes .csv extension
        ingest_db(file_path, table_name, creates)



(206529, 9)
(224489, 9)
(2372474, 16)
(12261, 9)


In [ ]:
conn = sqlite3.connect("shops.db")
tables = pd.read_sql_query("SELECT name FROM sqlite_master where type='table'", conn)
print(tables)


In [ ]:
for table in tables["name"]:
    print("-"*30, f'{table}','-'*50)
    print(pd.read_sql_query(f"SELECT count(*) FROM  {table} limit 5",conn))
    display(pd.read_sql_query(f"SELECT * FROM  {table} limit 5",conn))

In [ ]:
purchases = pd.read_sql("select * from purchases where VendorNumber = 4466",conn)
purchases

In [ ]:
purchase_prices = pd.read_sql("select * from purchase_prices where VendorNumber = 4466 " ,conn)
purchase_prices


In [ ]:
vendor_invoice = pd.read_sql("select * from vendor_invoice where VendorNumber = 4466", conn)
vendor_invoice

In [ ]:
sales = pd.read_sql("select * from sales where vendorNo = 4466",conn)
sales

In [ ]:
purchases

In [ ]:
purchases.groupby(["Brand","PurchasePrice"])[["Quantity","Dollars"]].sum()

In [ ]:
purchase_prices

In [ ]:
vendor_invoice["PONumber"].nunique()

In [ ]:
vendor_invoice

In [ ]:
vendor_invoice["VendorName"]

In [ ]:
sales.groupby("Brand")[["SalesDollars", "SalesPrice", "SalesQuantity"]].sum()

*   The purchases table contains actual purchase data, including the date of purchase, products (brands) purchased by vendors, the amount paid (in dollars), and the quantity purchased.
*   The purchase_price column is derived from the purchase_prices table, which provides product-wise actual and purchase prices. The combination of vendor and brand is unique in this table.
*   The vendor_invoice table aggregates data from the purchases table, summarizing quantity and dollar amounts, along with an additional column for freight. This table maintains uniqueness based on vendor and PO number.
*   The sales table captures actual sales transactions, detailing the brands purchased by vendors, the quantity sold, the selling price, and the revenue earned.
___________________________________________________________________________________________
As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:

*   purchase transactions made by vendors
*   sales transaction data
*   freight costs for each vendor
*   actual product prices from vendors



In [ ]:
vendor_invoice.columns

In [ ]:
vendor_invoice["PONumber"]

In [ ]:
freight_summary = pd.read_sql_query("""select vendorNumber, sum(Freight) as FreightCost from vendor_invoice
                                    group by VendorNumber""", conn)
freight_summary

In [ ]:
purchases.columns

In [ ]:
purchase_prices.columns

In [ ]:
pd.read_sql("""select
   p.VendorNumber,
   p.VendorName,
   P.Brand,
   p.PurchasePrice,
   pp.Volume,
   pp.Price as ActualPrice,
   sum(p.Quantity) as TotalPurchaseQuantity,
   sum(p.Dollars) as TotalPurchaseDollars
   from purchases as p
   join purchase_prices as pp
   on p.Brand = pp.Brand
   group by p.VendorNumber , p.VendorName, p.Brand
   having p.PurchasePrice > 0
   order by TotalPurchaseDollars""", conn)

In [ ]:
sales.columns
sales

In [ ]:
Sales_Data = pd.read_sql("""select
   VendorNo,  
   Brand,
   sum(SalesDollars) as TotalSalesDollars,
   sum(SalesPrice) as TotalSalesPrice,
   sum(SalesQuantity) as TotalSalesQuantity,
   sum(ExciseTax) as TotalExciseTax
   from sales
   group by VendorNo, Brand
   order by VendorNo asc""", conn);
Sales_Data

In [ ]:
import time

In [ ]:
vendor_sales_summary = pd.read_sql("""WITH FreightSummary as (
    select
        VendorNumber,
        sum(Freight) as FreightCost
    From
        Vendor_invoice 
    Group by
        VendorNumber
),

PurchaseSummary as( 
   select
       p.VendorNumber,
       p.VendorName,
       p.Brand,
       p.Description,
       p.PurchasePrice,
       pp.Price as ActualPrice,
       pp.Volume,
       sum(p.Quantity) as TotalPurchaseQuantity,
       sum(p.Dollars) as TotalPurchaseDollars
   from
       purchases as p
   join
       purchase_prices as pp on p.Brand = pp.Brand
   where
       p.PurchasePrice > 0
   group by
       p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary as (
    select
       VendorNo,
       Brand,
       sum(SalesDollars) as TotalSalesDollars,
       sum(SalesPrice) as TotalSalesPrice,
       sum(SalesQuantity) as TotalSalesQuantity,
       sum(ExciseTax) as TotalExciseTax
   from
       sales
   group by
       VendorNo, Brand
)

select
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,  
    ss.TotalSalesDollars,    
    ss.TotalSalesPrice,    
    ss.TotalExciseTax,       
    fs.FreightCost
from
    PurchaseSummary as ps 
    
    SalesSummary as ss on ps.VendorNumber = ss.VendorNo AND ps.Brand = ss.Brand
left join
    FreightSummary as fs on ps.VendorNumber = fs.VendorNumber
order by
    ps.TotalPurchaseDollars asc """, conn)



In [ ]:
vendor_sales_summary

This query generates a vendor-wise sales and purchase summary, which is valuable for:

**Performance Optimization:**
<br>
<b>.</b> The query involves heavy joins and aggregations on large datasets like sales and purchases.</br>
<b>.</b> Storing the pre-aggregated reults avoids repeated expensive computations.</br>
**.** Helps in analyzing sales,purchases,and pricing for different vendors and brands.</br>
**.** Future Benefits of storing this data for faster Dashboarding & Reporting.</br>
**.** Instead of running expensive queries each time,dashboards can fetch data quickly from vendor_sales_summary.


In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary.head()

In [ ]:
vendor_sales_summary["VendorName"]

In [ ]:
vendor_sales_summary["VendorName"].str.strip()
vendor_sales_summary["Description"].str.strip()

In [ ]:
vendor_sales_summary["Volume"] = vendor_sales_summary["Volume"].astype('float64')

In [ ]:
vendor_sales_summary["Volume"]

In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary.fillna(0, inplace = True)

In [ ]:
vendor_sales_summary["GrossProfit"] = vendor_sales_summary["TotalSalesDollars"] - vendor_sales_summary["TotalPurchaseDollars"]

In [ ]:
vendor_sales_summary["ProfitMargin"] = (vendor_sales_summary["GrossProfit"] / vendor_sales_summary["TotalSalesDollars"])*100

In [ ]:
vendor_sales_summary["StockTurnover"] = vendor_sales_summary["TotalSalesQuantity"]/vendor_sales_summary["TotalPurchaseQuantity"]

In [ ]:
vendor_sales_summary["SalestoPurchaseRatio"] = vendor_sales_summary["TotalSalesDollars"]/vendor_sales_summary["TotalPurchaseDollars"]

In [ ]:
vendor_sales_summary

In [ ]:
cursor = conn.cursor()

In [ ]:
vendor_sales_summary.columns

In [ ]:
cursor.execute("""create table vendor_sales_summary (
       VendorNumber int,
       VendorName varchar(100),
       Brand int,
       Description varchar(100),
       ActualPrice decimal(10,2),
       Volume ,
       TotalPurchaseQuantity int,
       TotalPurchaseDollars decimal(15,2),
       TotalSalesQuantity int,
       TotalSalesDollars decimal(15,2),
       TotalSalesPrice decimal(15,2),
       TotalExciseTax decimal(15,2),
       FreightCost decimal(15,2),
       GrossProfit decimal(15,2),
       ProfitMargin decimal(15,2),
       StockTurnover decimal(15,2),
       SaleestoPurchaseRatio decimal(15,2),
       SalestoPurchaseRatio decimal(15,2),
       primary key (VendorNumber, Brand)
       );""")


In [ ]:
pd.read_sql_query("select * from vendor_sales_summary",conn)

In [ ]:
vendor_sales_summary.to_sql("vendor_sales_summary",conn, if_exists='replace', index = False)

In [27]:
vendor_sales_summary

NameError: name 'vendor_sales_summary' is not defined

In [28]:
conn

In [29]:
pd.read_sql_query("select * from vendor_sales_summary",conn)

,VendorNumber,VendorName,Brand,Description,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SaleestoPurchaseRatio,SalestoPurchaseRatio
0,7245,PROXIMO SPIRITS INC.,3065,Three Olives Grape Vodka,0.99,50.0,1,0.71,86.0,85.14,33.66,4.46,38994.78,84.43,99.166079,86.000000,119.915493,119.915493
1,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.99,200.0,1,1.47,72.0,143.28,77.61,15.12,257032.07,141.81,98.974037,72.000000,97.469388,97.469388
2,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.99,50.0,2,1.48,2.0,1.98,0.99,0.10,14069.87,0.50,25.252525,1.000000,1.337838,1.337838
3,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.49,50.0,6,2.34,134.0,65.66,1.47,7.04,50293.62,63.32,96.436186,22.333333,28.059829,28.059829
4,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,4.99,750.0,2,2.64,5.0,15.95,10.96,0.55,27100.41,13.31,83.448276,2.500000,6.041667,6.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,29.99,1750.0,138109,3023206.01,135838.0,4223107.62,545778.28,249587.83,257032.07,1199901.61,28.412764,0.983556,1.396897,1.396897
10688,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,22.99,1750.0,201682,3261197.94,200412.0,4475972.88,420050.01,368242.80,257032.07,1214774.94,27.139908,0.993703,1.372493,1.372493
10689,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,24.99,1750.0,187407,3418303.68,187140.0,4538120.60,461140.15,343854.07,123780.22,1119816.92,24.675786,0.998575,1.327594,1.327594
10690,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,28.99,1750.0,164038,3804041.22,160247.0,4819073.49,561512.37,294438.66,144929.24,1015032.27,21.062810,0.976890,1.266830,1.266830
